## Online Retailer Product Reviews Text Feature Extraction

In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade pip --pre

Note: you may need to restart the kernel to use updated packages.


In [6]:
%conda env create --name sdc python=3.8


SpecNotFound: Invalid name, try the format: user/package


Note: you may need to restart the kernel to use updated packages.


In [7]:
%conda activate sdc


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [8]:
%conda install ipykernel

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [9]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col
from snowflake.snowpark.types import *
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re, string
import nltk
import json
from nltk.stem import SnowballStemmer

## Connecting to Snowflake Data Cloud to access Online Retailer Online Retailer Product Ratings and Reviews from the Snowflake Marketplace

In [10]:
with open('creds.json') as f:
    connection_parameters = json.load(f)    

session = Session.builder.configs(connection_parameters).create()
 
#sql = "SELECT REVIEW_COMMENTS from RETAILER_REVIEW_SENTIMENT"

In [11]:
%%time
df = session.table('RETAILER_REVIEW_SENTIMENT').filter(col("TOPIC")== "price").select(col("REVIEW_COMMENTS")).toPandas()

CPU times: user 1.32 s, sys: 258 ms, total: 1.58 s
Wall time: 5.82 s


## Prepare Text Cleaning Functions

In [12]:
df.head()

,REVIEW_COMMENTS
0,"great price, they were so hard to find in stor..."
1,GOOD PRICE AND FAST SHIPPING~
2,Used for Christmas decorations. Great value f...
3,These batteries were installed in a clock. The...
4,Great price!!! Always in my favorites to purc...


In [13]:
# Prepare cleaning functions

stop_words = ["a", "an", "the", "this", "that", "is", "it", "to", "and"]

stemmer = SnowballStemmer('english')

def preProcessText(text):
    # lowercase and strip leading/trailing white space
 
    text = text.lower().strip()
    
    # remove HTML tags
    text = re.compile('<.*?>').sub('', text)
    
    # remove punctuation
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    
    # remove extra white space
    text = re.sub('\s+', ' ', text)
    
    return text

def lexiconProcess(text, stop_words, stemmer):
    filtered_sentence = []
    words = text.split(" ")
    for w in words:
        if w not in stop_words:
            filtered_sentence.append(stemmer.stem(w))
    text = " ".join(filtered_sentence)
    
    return text

def cleanSentence(text, stop_words, stemmer):
    return lexiconProcess(preProcessText(text), stop_words, stemmer)

## Prepare vectorizer

In [14]:
 
from sklearn.feature_extraction.text import CountVectorizer

textvectorizer = CountVectorizer(binary=True)# can also limit vocabulary size here, with say, max_features=50

## Clean and vectorize a text feature with the Snowflake Data Marketplace Online Retailer Comments Data

In [15]:
import numpy as np
#text_feature = np.array(df.to_numpy())
text_feature = np.array([ elem for singleList in df.to_numpy() for elem in singleList])
print(len(text_feature))

# Clean up the text
text_feature_cleaned = [cleanSentence(item, stop_words, stemmer) for item in text_feature]

# Vectorize the cleaned text
text_feature_vectorized = textvectorizer.fit_transform(text_feature_cleaned)
print('Vocabulary: \n', textvectorizer.vocabulary_)
print('Bag of Words Binary Features: \n', text_feature_vectorized.toarray())

print(text_feature_vectorized.shape)

31849
Vocabulary: 
 {'great': 3978, 'price': 6432, 'they': 8122, 'were': 8862, 'so': 7504, 'hard': 4094, 'find': 3535, 'in': 4429, 'store': 7752, 'was': 8802, 'eleg': 3145, 'solut': 7532, 'good': 3917, 'fast': 3451, 'ship': 7295, 'use': 8592, 'for': 3648, 'christma': 2141, 'decor': 2677, 'valu': 8625, 'these': 8113, 'batteri': 1461, 'instal': 4524, 'clock': 2195, 'work': 8976, 'fine': 3537, 'veri': 8663, 'reason': 6730, 'deliveri': 2723, 'on': 5864, 'time': 8203, 'alway': 1116, 'my': 5592, 'favorit': 3469, 'purchas': 6562, 'hallmark': 4062, 'tree': 8344, 'ornament': 5925, 'need': 5640, 'light': 4974, 'up': 8558, 'play': 6272, 'music': 5579, 'have': 4120, 'realli': 6724, 'last': 4857, 'kept': 4734, 'littl': 5019, 'live': 5020, 'paid': 6045, 'afraid': 996, 'would': 9002, 'receiv': 6749, 'expir': 3363, 'or': 5904, 'about': 894, 'be': 1521, 'but': 1856, 'definit': 2698, 'not': 5744, 'case': 1973, 'date': 2619, 'year': 9058, 'from': 3737, '2019': 252, 'am': 1119, 'super': 7874, 'happi': 408